In [1]:
from re import T
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras import layers

In [2]:

# From: https://keras.io/examples/structured_data/structured_data_classification_from_scratch/

dataframe = pd.read_csv("./test.csv", keep_default_na=False)

print(dataframe.shape)

val_dataframe = dataframe.sample(frac=0.2, random_state=1337)
train_dataframe = dataframe.drop(val_dataframe.index)

print(
    "Using %d samples for training and %d for validation"
    % (len(train_dataframe), len(val_dataframe))
)

mlb = LabelBinarizer()
def dataframe_to_dataset(dataframe):
    dataframe = dataframe.copy()
    dataframe.pop("name")
    labels = dataframe.pop("tier")
    labels = mlb.fit_transform(labels)
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    ds = ds.shuffle(buffer_size=len(dataframe))
    return ds

train_ds = dataframe_to_dataset(train_dataframe)
val_ds = dataframe_to_dataset(val_dataframe)

for x, y in train_ds.take(1):
    print("Input:", x)
    print("Target:", y)

(677, 13)
Using 542 samples for training and 135 for validation
Input: {'hp': <tf.Tensor: shape=(), dtype=int64, numpy=80>, 'atk': <tf.Tensor: shape=(), dtype=int64, numpy=135>, 'def': <tf.Tensor: shape=(), dtype=int64, numpy=130>, 'spa': <tf.Tensor: shape=(), dtype=int64, numpy=95>, 'spd': <tf.Tensor: shape=(), dtype=int64, numpy=90>, 'speed': <tf.Tensor: shape=(), dtype=int64, numpy=70>, 'ability1': <tf.Tensor: shape=(), dtype=string, numpy=b'Clear Body'>, 'ability2': <tf.Tensor: shape=(), dtype=string, numpy=b''>, 'ability3': <tf.Tensor: shape=(), dtype=string, numpy=b'Light Metal'>, 'type1': <tf.Tensor: shape=(), dtype=string, numpy=b'Steel'>, 'type2': <tf.Tensor: shape=(), dtype=string, numpy=b'Psychic'>}
Target: tf.Tensor([0 0 0 0 1 0 0 0], shape=(8,), dtype=int32)


In [3]:

train_ds = train_ds.batch(32)
val_ds = val_ds.batch(32)

from tensorflow.keras.layers import IntegerLookup
from tensorflow.keras.layers import Normalization
from tensorflow.keras.layers import StringLookup

def encode_numerical_feature(feature, name, dataset):
    # Create a Normalization layer for our feature
    normalizer = Normalization()

    # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # Learn the statistics of the data
    normalizer.adapt(feature_ds)

    # Normalize the input feature
    encoded_feature = normalizer(feature)
    return encoded_feature


def encode_categorical_feature(feature, name, dataset, is_string):
    lookup_class = StringLookup if is_string else IntegerLookup
    # Create a lookup layer which will turn strings into integer indices
    lookup = lookup_class(output_mode="binary")

    # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # Learn the set of possible string values and assign them a fixed integer index
    lookup.adapt(feature_ds)

    # Turn the string input into integer indices
    encoded_feature = lookup(feature)
    return encoded_feature

# Categorical features encoded as integers
hp = keras.Input(shape=(1,), name="hp", dtype="int64")
atk = keras.Input(shape=(1,), name="atk", dtype="int64")
defe = keras.Input(shape=(1,), name="def", dtype="int64")
spa = keras.Input(shape=(1,), name="spa", dtype="int64")
spd = keras.Input(shape=(1,), name="spd", dtype="int64")
speed = keras.Input(shape=(1,), name="speed", dtype="int64")
#strongestAttack = keras.Input(shape=(1,), name="strongestAttack", dtype="int64")
#recovery = keras.Input(shape=(1,), name="recovery", dtype="int64")
#coverageAttacks = keras.Input(shape=(1,), name="coverageAttacks", dtype="int64")
prevo = keras.Input(shape=(1,), name="prevo", dtype="int64")

# Categorical feature encoded as string
ability1 = keras.Input(shape=(1,), name="ability1", dtype="string")
ability2 = keras.Input(shape=(1,), name="ability2", dtype="string")
ability3 = keras.Input(shape=(1,), name="ability3", dtype="string")
type1 = keras.Input(shape=(1,), name="type1", dtype="string")
type2 = keras.Input(shape=(1,), name="type2", dtype="string")


all_inputs = [
    hp,
    atk,
    defe,
    spa,
    spd,
    speed,
    #strongestAttack,
    #recovery,
    #coverageAttacks,
    ##prevo,
    ability1,
    ability2,
    ability3,
    type1,
    type2,
]

# Integer categorical features
#recovery_encoded = encode_categorical_feature(recovery, "recovery", train_ds, False)
#prevo_encoded = encode_categorical_feature(prevo, "prevo", train_ds, False)

# String categorical features
ability1_encoded = encode_categorical_feature(ability1, "ability1", train_ds, True)
ability2_encoded = encode_categorical_feature(ability2, "ability2", train_ds, True)
ability3_encoded = encode_categorical_feature(ability3, "ability3", train_ds, True)
type1_encoded = encode_categorical_feature(type1, "type1", train_ds, True)
type2_encoded = encode_categorical_feature(type2, "type2", train_ds, True)

# Numerical features
hp_encoded = encode_numerical_feature(hp, "hp", train_ds)
atk_encoded = encode_numerical_feature(atk, "atk", train_ds)
defe_encoded = encode_numerical_feature(defe, "def", train_ds)
spa_encoded = encode_numerical_feature(spa, "spa", train_ds)
spd_encoded = encode_numerical_feature(spd, "spd", train_ds)
speed_encoded = encode_numerical_feature(speed, "speed", train_ds)
#strongestAttack_encoded = encode_numerical_feature(strongestAttack, "strongestAttack", train_ds)
#coverageAttacks_encoded = encode_numerical_feature(coverageAttacks, "coverageAttacks", train_ds)

all_features = layers.concatenate(
    [
        hp_encoded,
        atk_encoded,
        defe_encoded,
        spa_encoded,
        spd_encoded,
        speed_encoded,
        #strongestAttack_encoded,
        #recovery_encoded,
        #coverageAttacks_encoded,
        #prevo_encoded,
        ability1_encoded,
        ability2_encoded,
        ability3_encoded,
        type1_encoded,
        type2_encoded,
    ]
)
x = layers.Dense(32, activation="relu")(all_features)
x = layers.Dense(32, activation="relu")(x)
x = layers.Dense(32, activation="relu")(x)
x = layers.Dense(32, activation="relu")(x)
x = layers.Dense(32, activation="relu")(x)
x = layers.Dropout(0.5)(x)
output = layers.Dense(len(mlb.classes_), activation="sigmoid")(x)
model = keras.Model(all_inputs, output)
model.compile("adam", "binary_crossentropy", metrics=["accuracy"])


In [4]:

model.fit(train_ds, epochs=600, validation_data=val_ds)


Epoch 1/600
17/17 [==============================] - 1s 19ms/step - loss: 0.6814 - accuracy: 0.1402 - val_loss: 0.6640 - val_accuracy: 0.2741
Epoch 2/600
17/17 [==============================] - 0s 8ms/step - loss: 0.6333 - accuracy: 0.1587 - val_loss: 0.5763 - val_accuracy: 0.2963
Epoch 3/600
17/17 [==============================] - 0s 8ms/step - loss: 0.5202 - accuracy: 0.1642 - val_loss: 0.4113 - val_accuracy: 0.2741
Epoch 4/600
17/17 [==============================] - 0s 8ms/step - loss: 0.4354 - accuracy: 0.2232 - val_loss: 0.3509 - val_accuracy: 0.2593
Epoch 5/600
17/17 [==============================] - 0s 8ms/step - loss: 0.3891 - accuracy: 0.2823 - val_loss: 0.3301 - val_accuracy: 0.2889
Epoch 6/600
17/17 [==============================] - 0s 8ms/step - loss: 0.3356 - accuracy: 0.3672 - val_loss: 0.3107 - val_accuracy: 0.3259
Epoch 7/600
17/17 [==============================] - 0s 8ms/step - loss: 0.3169 - accuracy: 0.4133 - val_loss: 0.2941 - val_accuracy: 0.4593
Epoch 8/600


In [5]:
# Dragapult
sample = {
    "hp": 88,
    "atk": 120,
    "def": 75,
    "spa": 100,
    "spd": 75,
    "speed": 142,
    "ability1": "Clear Body",
    "ability2": "Infiltrator",
    "ability3": "Cursed Body",
    "type1": "Dragon",
    "type2": "Ghost",
    #"strongestAttack": 130,
    #"recovery": 0,
    #"coverageAttacks": 12,
    #"prevo": 0
}

input_dict = {name: tf.convert_to_tensor([value]) for name, value in sample.items()}
predictions = model.predict(input_dict)

print(predictions)
print(mlb.classes_)
print(mlb.classes_[np.argmax(predictions)])

[[2.0273667e-22 0.0000000e+00 9.9379224e-01 0.0000000e+00 1.2602389e-13
  2.0065544e-22 1.9785194e-25 0.0000000e+00]]
['LC' 'NU' 'OU' 'PU' 'RU' 'UU' 'Uber' 'ZU']
OU


In [6]:
dataframe = pd.read_csv("./hisui.csv", keep_default_na=False)

for index, entry in dataframe.iterrows():
    print(entry["name"])
    input_dict = {name: tf.convert_to_tensor([value]) for name, value in entry.items() if name != 'tier' and name != 'name'}
    
    predictions = model.predict(input_dict)

    print(predictions)
    print(mlb.classes_)
    print(mlb.classes_[np.argmax(predictions)])
    print()

Growlithe-Hisui
[[1. 0. 0. 0. 0. 0. 0. 0.]]
['LC' 'NU' 'OU' 'PU' 'RU' 'UU' 'Uber' 'ZU']
LC

Arcanine-Hisui
[[1.8011942e-33 0.0000000e+00 0.0000000e+00 1.9007439e-07 4.7470908e-25
  9.9454767e-01 1.4276599e-31 3.4663767e-24]]
['LC' 'NU' 'OU' 'PU' 'RU' 'UU' 'Uber' 'ZU']
UU

Voltorb-Hisui
[[1. 0. 0. 0. 0. 0. 0. 0.]]
['LC' 'NU' 'OU' 'PU' 'RU' 'UU' 'Uber' 'ZU']
LC

Electrode-Hisui
[[1.2866423e-32 2.1174442e-07 1.0964530e-20 1.6024434e-25 3.3684086e-05
  8.7163190e-27 3.1904010e-34 0.0000000e+00]]
['LC' 'NU' 'OU' 'PU' 'RU' 'UU' 'Uber' 'ZU']
RU

Typhlosion-Hisui
[[2.0986835e-22 5.6528471e-10 1.0396371e-15 3.6180808e-08 9.4743091e-01
  1.6038501e-10 3.3923117e-23 4.0341357e-27]]
['LC' 'NU' 'OU' 'PU' 'RU' 'UU' 'Uber' 'ZU']
RU

Qwilfish-Hisui
[[9.0337621e-24 0.0000000e+00 0.0000000e+00 6.3427029e-23 0.0000000e+00
  1.4435647e-29 0.0000000e+00 9.9998939e-01]]
['LC' 'NU' 'OU' 'PU' 'RU' 'UU' 'Uber' 'ZU']
ZU

Sneasel-Hisui
[[3.1416965e-17 2.3073815e-02 3.8263771e-16 2.0722118e-03 7.0267998e-02
  4.3